In [1]:
from kaggle_environments import evaluate, make, utils
from IPython.display import clear_output
import numpy as np
import random as rand
import pandas as pd
import matplotlib.pyplot as plt

env = make("connectx", debug=True)
env.render()

Loading environment lux_ai_2022 failed: No module named 'vec_noise'


In [2]:
from NN_architecture import PyCNNik_A4
from DeepQAgent_torch import QTorchModelClass, CreateDatasets

2023-02-22 13:09:19.769703: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /var/lib/cdsw-custom/addons/lib64
2023-02-22 13:09:19.769732: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/home/cdsw/.local/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
DQNI = QTorchModelClass(model_name ="allstar_A4_torch_04", exploration_factor=0.9, alpha= 0.5, early_stopping = False, configuration= env.configuration, model_architecture = PyCNNik_A4)
DQNI.load_model()

loading modelDQNs/allstar_A4_torch_04.h5


In [4]:
def dqn_agent(observation, configuration):
    
    board_state = DQNI.obs_to_board_state(observation)
    #epsilon greedy policy
    p = rand.uniform(0, 1)

    if p < DQNI.exp_factor:
        #exploitation move
        move = DQNI.current_optimal_move(board_state)
    else:
        #exploration move
        all_options = DQNI.all_options(board_state)
        move = int(rand.choice(all_options))
            
    return move

In [5]:
def dqn_field_agent(observation, configuration):
    
    board_state = DQNI.obs_to_board_state(observation)
    
    move = DQNI.current_optimal_move(board_state)
            
    return move

In [6]:
from agent_rule_based import rule_based_agent
from negamax import negamax_agent

In [7]:
DQNI.reset_memory()

In [ ]:
################################### training agent 
########################################################################################################################################################
# number of epochs
epochs_n = 100000
# after how many epochs do we train
training_batch = 64
rewards = []

for epoch in range(1, epochs_n + 1):
    
    print(f"Epoch: {epoch}")

    j = np.random.randint(1)
    l = np.random.randint(2)
    if j == 0:
        if l == 0:
            AGENT = negamax_agent
            trainer = env.train([None, dqn_agent])
            print("negamax vs dqn_agent")
            r = float(-1)
        if l == 1:
            AGENT = dqn_agent
            trainer = env.train([None, "negamax"])
            print("dqn_agent vs negamax")
            r = float(1)

    observation = trainer.reset()
    done = False
    move_count = 0
    
    while not done:
        
        board_state = DQNI.obs_to_board_state(env.state[0].observation)
        
        move_count += 1
        if move_count > 1:
            DQNI.load_to_memory(DQNI.prev_state, DQNI.prev_move, board_state, DQNI.reward(env.state[0].status, env.state[0].reward))
        if move_count > 2:
            #load opponents experience to memory
            DQNI.load_opponents_experience_to_memory(DQNI.past_prev_state, DQNI.past_prev_move, DQNI.prev_state, DQNI.prev_move, 
                                                DQNI.prev_reward_opponent)

        my_action = AGENT(env.state[0].observation, env.configuration)
            
        observation, reward, done, info = trainer.step(my_action)
        
        DQNI.past_prev_state = DQNI.prev_state
        DQNI.past_prev_move = DQNI.prev_move
        DQNI.prev_state = board_state
        DQNI.prev_move = my_action
        DQNI.prev_reward = env.state[0].reward
        DQNI.prev_reward_opponent = env.state[-1].reward
        
    board_state_game_finish = DQNI.obs_to_board_state(env.state[0].observation)
    DQNI.load_to_memory(DQNI.prev_state, DQNI.prev_move, board_state_game_finish, DQNI.reward(env.state[0].status, env.state[0].reward))
    
    #load opponents experience to memory
    #print(np.count_nonzero(DQMInstance.prev_state-board_state_game_finish))
    if np.count_nonzero(DQNI.prev_state-board_state_game_finish) <= 1:
        DQNI.load_opponents_experience_to_memory(DQNI.past_prev_state, DQNI.past_prev_move, DQNI.prev_state, DQNI.prev_move, env.state[-1].reward)
    if np.count_nonzero(DQNI.prev_state-board_state_game_finish) == 2:
        DQNI.load_opponents_experience_to_memory(DQNI.past_prev_state, DQNI.past_prev_move, DQNI.prev_state, DQNI.prev_move, 0)
        board_state_1 = DQNI.make_state_from_move(DQNI.prev_state, DQNI.prev_move)
        board_state_2 = board_state_game_finish
        opp_move = DQNI.opponent_move_from_states(board_state_1, board_state_2)
        DQNI.memory.append([DQNI.switch_sides(board_state_1), opp_move, DQNI.switch_sides(board_state_2), env.state[-1].reward])

    print(f"Reward for dqn_agent: {reward*r}")
    rewards.append(reward*r)
    
    # Offline (Batch) training
    if epoch % training_batch == 0:
        #create torch dataset
        training_batch_ds = CreateDatasets(DQNI, DQNI.memory)
        # fit
        DQNI.learn_batch(training_batch_ds)
        # save model after every training (training may crash)
        DQNI.save_model()
        # empty memory after every batch or let memory accumulate for given number of epochs
        if epoch % 320 == 0:
            DQNI.reset_memory()
        # save rewards for this batch of epochs
        DQNI.save_rewards(rewards)
        rewards = []

    
    clear_output(wait=True)

In [ ]:
reward_means = [] 
batches = []

for batch in range(1, 158):
    rm = pd.read_pickle("data/rewards_list_allstar_A4_torch_04_" + str(batch) + ".pkl")
    reward_means.append(np.mean(rm))
    batches.append(batch)
    
plt.title("Avg. Reward; Pretrained Deep-Q-Agent CNN_A4 03 vs. Negamax; Batches of 32")
plt.plot(batches,reward_means)

plt.savefig('data/Deep-Q-Agent CNN_A4 03 vs. Negamax.jpg',bbox_inches='tight', dpi=150)

In [ ]:
evaluate("connectx", [dqn_field_agent, "negamax"], num_episodes=20)

In [ ]:
evaluate("connectx", ["negamax", dqn_field_agent], num_episodes=20)